In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import funEnsemble
import funEnsemble.functional_ensemble_clustering as ec
import fda_results as fr
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
from FDApy.clustering.fcubt import Node, FCUBT
from FDApy.representation.functional_data import DenseFunctionalData
import os

os.environ["R_HOME"] = r"C:\Program Files\R\R-4.2.2" # change as needed

%reload_ext rpy2.ipython

# Load packages
%R library(fda)
%R library(funHDDC)
%R library(funFEM)
%R library(mclust)
%R library(tidyverse)
%R library(rlang)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Define the exponentiated quadratic 
def exponentiated_quadratic(xa, xb):
    # L2 distance (Squared Euclidian) where σ = 1
    sq_norm = -0.5 * scipy.spatial.distance.cdist(xa, xb, 'sqeuclidean')
    return np.exp(sq_norm)

R[write to console]: 
Attaching package: 'rlang'


R[write to console]: The following objects are masked from 'package:purrr':

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice




In [70]:
def scenario_one_simulate():

    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 8, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Create different curves
    mean_curve = np.zeros((4, len(t)))
    mean_curve[0] = np.reshape(-8*np.sin(t)*np.log(t+2), (nb_of_samples))
    mean_curve[1] = np.reshape(4*np.cos(t)*np.log(t+0.5), (nb_of_samples))
    mean_curve[2] = np.reshape(-2-4*np.cos(t)*(np.sqrt(5*t+0.5)), (nb_of_samples))
    mean_curve[3] = np.reshape(5*np.cos(2*t)*np.log(t+0.5)*(np.sqrt(t+0.5)), (nb_of_samples))

    mixture_k = [0.2, 0.2, 0.5, 0.1]
    n = 100
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[0], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0

        elif r < mixture_k[0] + mixture_k[1]:
            # draw for the second component
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[1], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1
        
        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2]:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[2], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2

        else:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[3], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 3

    return Y, simulation_label

In [77]:
def scenario_two_simulate():

    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 8, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Create different curves
    mean_curve = np.zeros((4, len(t)))
    mean_curve[0] = np.reshape(np.sin(t)*np.log(t+2), (nb_of_samples))
    mean_curve[1] = np.reshape(np.cos(t)*np.log(t+0.5), (nb_of_samples))
    mean_curve[2] = np.reshape(np.cos(t)*(np.sqrt(5*t+0.5)), (nb_of_samples))
    mean_curve[3] = np.reshape(np.cos(2*t)*np.log(t+0.5)*(np.sqrt(t+0.5)), (nb_of_samples))

    mixture_k = [1/4, 1/4, 1/4, 1/4	]
    n = 500
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[0], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0

        elif r < mixture_k[0] + mixture_k[1]:
            # draw for the second component
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[1], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1


        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2]:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[2], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2

        else:
            # draw for the first component
            ys = np.random.multivariate_normal(
            mean=mean_curve[3], cov=Σ, 
            size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 3

    return Y, simulation_label
    

In [ ]:
def scenario_three_simulate():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 20, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Declare the mean curve
    mean_curve = np.zeros((2, len(t)))
    # Mean curve with white noise
    mean_curve[0] = np.reshape(0.25+np.sin(t), (nb_of_samples))
    mean_curve[1] = np.reshape(np.cos(t), (nb_of_samples))

    mixture_k = [0.5, 0.5]
    n = 100
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0
        else:
            # draw for the first component
            ys = np.random.multivariate_normal(mean=mean_curve[1], cov=Σ, size=1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1
            
    return Y, simulation_label

        

In [ ]:
def scenario_four_simulate():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 20, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Declare the mean curve
    mean_curve = np.zeros((2, len(t)))
    # Mean curve with white noise
    mean_curve[0] = np.reshape(6 - np.abs(t - 6), (nb_of_samples))
    mean_curve[1] = np.reshape(6 - np.abs(t - 9), (nb_of_samples))

    mixture_k = [0.6, 0.4]
    n = 100
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = -1+0.25*np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1) + 0.5*np.random.multivariate_normal(mean=mean_curve[1], cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0
        else:
            # draw for the first component
            ys = 0.5*np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1) + 0.25*np.random.multivariate_normal(mean=mean_curve[1], cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1

    return Y, simulation_label


In [ ]:
def scenario_five_simulate():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 20, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t/0.5, t/12)  # Kernel of data points
    Σ2 = exponentiated_quadratic(t, t/12)  # Kernel of data points
    Σ3 = exponentiated_quadratic(t, t*2/3)  # Kernel of data points

    # Declare the mean curve
    mean_curve = np.zeros((3, len(t)))
    # Mean curve with white noise
    mean_curve[0] = np.reshape(6 - np.abs(t - 11), (nb_of_samples))
    mean_curve[1] = np.reshape(6 - np.abs(t - 7), (nb_of_samples))
    mean_curve[2] = np.reshape(6 - np.abs(t - 15), (nb_of_samples))


    mixture_k = [0.2, 0.5, 0.1, 0.4]
    n = 500
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = np.reshape(t/2, (nb_of_samples)) + np.random.multivariate_normal(mean=mean_curve[2], cov=Σ2, size=1) + np.random.multivariate_normal(mean=mean_curve[1], cov=Σ3, size=1) + np.random.normal(0, 0.125, len(t))*np.sqrt(0.1)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0
        elif r < mixture_k[0] + mixture_k[1]:
            # draw for the first component
            ys = np.reshape(t/2, (nb_of_samples)) + np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1) + np.random.multivariate_normal(mean=mean_curve[1], cov=Σ2, size=1)  + np.random.normal(0, 0.125, len(t))*np.sqrt(0.5)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1
        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2]:
            # draw for the first component
            ys = np.random.multivariate_normal(mean=mean_curve[1], cov=Σ3, size=1)+ np.random.normal(0, 0.125, len(t))*np.sqrt(10)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2
        else:
            # draw for the first component
            ys = np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1) + np.random.multivariate_normal(mean=mean_curve[2], cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))*np.sqrt(0.5)
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 3

    return Y, simulation_label


In [ ]:
def scenario_six_simulate():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 50, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data point
    Σ2 = exponentiated_quadratic(5*t, 5*t)  # Kernel of data point
    Σ3 = exponentiated_quadratic(8*t, 8*t)  # Kernel of data point

    mixture_k = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
    n = 200
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            ys = np.reshape(t, (nb_of_samples)) + np.random.multivariate_normal(mean=np.reshape(np.cos(t/10), (nb_of_samples)), cov=Σ, size=1) + np.random.multivariate_normal(mean=np.reshape(np.sin(1 + t/10), (nb_of_samples)), cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0
        elif r < mixture_k[0] + mixture_k[1]:
            ys = np.reshape(t, (nb_of_samples)) + 2*np.random.multivariate_normal(mean=np.reshape(np.cos(2*t/10), (nb_of_samples)), cov=Σ, size=1) + 2*np.random.multivariate_normal(mean=np.reshape(np.sin(2 + t/10), (nb_of_samples)), cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1
        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2]:
            ys = np.reshape(t, (nb_of_samples)) + 3*np.random.multivariate_normal(mean=np.reshape(np.cos(3*t/10), (nb_of_samples)), cov=Σ, size=1) + 3*np.random.multivariate_normal(mean=np.reshape(np.sin(3 + t/10), (nb_of_samples)), cov=Σ, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2
        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2] + mixture_k[3]:
            ys = np.reshape(t, (nb_of_samples)) + np.random.multivariate_normal(mean=np.reshape(np.sin(t/10), (nb_of_samples)), cov=Σ, size=1) + np.random.multivariate_normal(mean=np.reshape(np.cos(1 + t/10), (nb_of_samples)), cov=Σ2, size=1) + np.random.multivariate_normal(mean=np.reshape(np.square(t/10)+t/10, (nb_of_samples)), cov=Σ3, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 3
        elif r < mixture_k[0] + mixture_k[1] + mixture_k[2] + mixture_k[3] + mixture_k[4]:
            ys = np.reshape(t, (nb_of_samples))  + 2*np.random.multivariate_normal(mean=np.reshape(np.sin(2*t/10), (nb_of_samples)), cov=Σ, size=1) + 2*np.random.multivariate_normal(mean=np.reshape(np.cos(2 + t/10), (nb_of_samples)), cov=Σ2, size=1) + 2*np.random.multivariate_normal(mean=np.reshape(np.square(t/10)+t/10, (nb_of_samples)), cov=Σ3, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 4
        else:
            ys = np.reshape(t, (nb_of_samples)) + 3*np.random.multivariate_normal(mean=np.reshape(np.sin(3*t/10), (nb_of_samples)), cov=Σ, size=1) + 3*np.random.multivariate_normal(mean=np.reshape(np.cos(3 + t/10), (nb_of_samples)), cov=Σ2, size=1) + 3*np.random.multivariate_normal(mean=np.reshape(np.square(t/10)+t/10, (nb_of_samples)), cov=Σ3, size=1) + np.random.normal(0, 0.125, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 5

    return Y, simulation_label

In [ ]:
def scenario_seven_simulate():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 20, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Declare the mean curve
    mean_curve = np.zeros((3, len(t)))
    # Mean curve with white noise
    mean_curve[0] = np.reshape(np.cos(t), (nb_of_samples))
    mean_curve[1] = np.reshape(np.cos(t) -0.5*t, (nb_of_samples))
    mean_curve[2] = np.reshape(-0.5*t+0.5, (nb_of_samples))

    mixture_k = [1/2, 1/4, 1/4]
    n = 100
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = 1 + np.random.multivariate_normal(mean= mean_curve[0], cov=Σ, size=1) + np.random.normal(0, 0.2, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0
        elif r < mixture_k[0] + mixture_k[1]:
            # draw for the first component
            ys = 2 + np.random.multivariate_normal(mean= mean_curve[1], cov=Σ, size=1) + np.random.normal(0, 0.2, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1
        else:
            # draw for the first component
            ys = np.random.multivariate_normal(mean= mean_curve[2], cov=Σ, size=1) + np.random.normal(0, 0.2, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2

    return Y, simulation_label

In [ ]:
def scenario_eight_simulation():
    # Sample from the Gaussian process distribution
    nb_of_samples = 30  # Number of points in each function
    # Independent variable samples
    t = np.expand_dims(np.linspace(0, 20, nb_of_samples), 1)
    Σ = exponentiated_quadratic(t, t)  # Kernel of data points

    # Create different curves
    mean_curve = np.zeros((3, len(t)))
    mean_curve[0] = np.reshape(np.cos(1.5*np.pi*t), (nb_of_samples))
    mean_curve[1] = np.reshape(np.sin(1.5*np.pi*t), (nb_of_samples))
    mean_curve[2] = np.reshape(np.sin(np.pi*t), (nb_of_samples))

    mixture_k = [0.2, 0.6, 0.2]
    n = 100
    Y = np.zeros((n, len(t)))
    simulation_label = np.zeros(n)

    for i in range(n):
        # generate a random number between 0 and 1
        r = np.random.rand()
        # find the index of the mixture component
        if r < mixture_k[0]:
            # draw for the first component
            ys = - 5 + np.reshape(0.5*t, (nb_of_samples)) + np.random.multivariate_normal(mean=mean_curve[0], cov=Σ, size=1) + np.random.normal(0, 1, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 0

        elif r < mixture_k[0] + mixture_k[1]:
            # draw for the second component
            # draw for the first component
            ys = 5 + np.reshape(-0.5*t, (nb_of_samples)) + np.random.multivariate_normal(mean=mean_curve[1], cov=Σ, size=1) + np.random.normal(0, 2, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 1

        else:
            # draw for the first component
            ys = np.random.multivariate_normal(mean=mean_curve[2], cov=Σ, size=1) + np.random.normal(0, 3, len(t))
            Y[i] = ys
            # Remember the label of the component
            simulation_label[i] = 2

    return Y, simulation_label


In [ ]:
def eigen_dimension_selection(variance):
    V = 0
    # Determine the Eigen Dimension that explains 95% of the variance
    for i in range(len(variance)):
        if sum(variance[:i]) >= 0.95:
            V = i - 1
            break
    if V == 0:
        V = 1
    return V

In [15]:
# Make t a dict (for fcubt)
t = np.expand_dims(np.linspace(0, 20, len(Y[0])), 1)
t = {'input_dim_0': t}
t = {k: v.squeeze() for k, v in t.items()}

In [78]:
runs = 25

AMI_funensemble, ARI_funensemble, AMI_fCUBT, ARI_fCUBT, AMI_funFEM, ARI_funFEM, AMI_funHDDC, ARI_funHDDC = 0, 0, 0, 0, 0, 0, 0, 0

for i in range(runs):
    
    # Get data from the first simulation
    Y, simulation_label = scenario_two_simulate()

    # Dimension selection
    variance, ami, ari, v_range = fr.percentage_variation_ami_ari(10, Y, 150, simulation_label, 4)
    dimension = eigen_dimension_selection(variance)	

    # funEnsemble
    data_smooth, mean, principal_componenets, eigen_functions = ec.functional_data_decomposition (Y, dimension, 150)
    membership_matrices, labels = ec.functional_data_clustering (principal_componenets, 4)
    AMI_funensemble += adjusted_mutual_info_score(simulation_label, labels)
    ARI_funensemble += adjusted_rand_score(simulation_label, labels)

    # fCUBT
    data = DenseFunctionalData(t, Y)
    # Build the tree
    root_node = Node(data, is_root=True)
    fcubt = FCUBT(root_node=root_node)
    # Growing
    fcubt.grow(n_components=0.95, min_size=10)
    # Joining
    fcubt.join(n_components=0.95)
    AMI_fCUBT += adjusted_mutual_info_score(simulation_label, fcubt.labels_join)
    ARI_fCUBT += adjusted_rand_score(simulation_label, fcubt.labels_join)

    # funHDDC
    # Store the data in DataStore.csv
    np.savetxt('../Data/DataStore.csv', Y, delimiter=',') 
    # Load data
    %R argvals <- seq(0, 1, length.out = 30)
    %R values <- t(as.matrix(read.csv('~/ProjectDocs/Project_code/simuations/Data/DataStore.csv', header = FALSE))) # nolint
    # FunHDDC
    %R basis <- create.bspline.basis(rangeval = c(min(argvals), max(argvals)), nbasis = 25, norder = 3) # nolint
    %R data_fd <- smooth.basis(argvals = argvals, y = values, fdParobj = basis)$fd
    %R res_clust <- funHDDC(data_fd, K = 4, threshold = 0.3, model = 'ABQkDk', itermax = 2000, eps = 1e-3, init = 'kmeans') # nolint
    %R pred_labels <- res_clust$class
    # Write the pred_labels to a file
    %R write.csv(pred_labels, file = '~/ProjectDocs/Project_code/simuations/Data/PredictedLabelsStore.csv') # nolint
    # Read Labels from PredictedLabels.csv, ignoring the header and the first column
    predicted_labels = np.genfromtxt('../Data/PredictedLabelsStore.csv', delimiter=',', skip_header=1, usecols=1)
    AMI_funHDDC += adjusted_mutual_info_score(simulation_label, predicted_labels)
    ARI_funHDDC += adjusted_rand_score(simulation_label, predicted_labels)

    # FunFEM
    %R res_clust = try(funFEM(data_fd,K=4)) # nolint # nolint: commas_linter.
    %R try(pred_labels_fem <- res_clust$P) # nolint
    # Write the pred_labels to a file
    %R write.csv(pred_labels_fem, file = '~/ProjectDocs/Project_code/simuations/Data/PredictedLabelsStore.csv') # nolint
    # Read labels from PredictedLabels.csv, ignoring the header and the first column and getting the first four columns
    predicted_labels = np.genfromtxt('../Data/PredictedLabelsStore.csv', delimiter=',', skip_header=1, usecols=range(1, 5))
    #   Format the labels
    labels = np.zeros(len(predicted_labels))
    for i in range(len(predicted_labels)):
        labels[i] = np.argmax(predicted_labels[i])
    AMI_funFEM += adjusted_mutual_info_score(simulation_label, labels)
    ARI_funFEM += adjusted_rand_score(simulation_label, labels)

    # Say that the run is done
    print("--------------- FINISHED RUN ---------------")

   model K threshold complexity           BIC
1 ABQKDK 4       0.3        564 -3,892,659.41

SELECTED: model  ABQKDK  with  4  clusters.
Selection Criterion: BIC.
--------------- FINISHED RUN ---------------
   model K threshold complexity         BIC
1 ABQKDK 4       0.3        478 -725,632.02

SELECTED: model  ABQKDK  with  4  clusters.
Selection Criterion: BIC.
--------------- FINISHED RUN ---------------
   model K threshold complexity           BIC
1 ABQKDK 4       0.3        503 -2,174,049.63

SELECTED: model  ABQKDK  with  4  clusters.
Selection Criterion: BIC.
--------------- FINISHED RUN ---------------
   model K threshold complexity         BIC
1 ABQKDK 4       0.3        353 -701,762.06

SELECTED: model  ABQKDK  with  4  clusters.
Selection Criterion: BIC.
--------------- FINISHED RUN ---------------
   model K threshold complexity           BIC
1 ABQKDK 4       0.3        485 -1,927,633.72

SELECTED: model  ABQKDK  with  4  clusters.
Selection Criterion: BIC.
-------------

In [75]:
# print("AMI_funEnsemble: ", AMI_funensemble/runs)
# print("ARI_funEnsemble: ", ARI_funensemble/runs)
# print("AMI_fCUBT: ", AMI_fCUBT/runs)
# print("ARI_fCUBT: ", ARI_fCUBT/runs)
# print("AMI_funHDDC: ", AMI_funHDDC/runs)
# print("ARI_funHDDC: ", ARI_funHDDC/runs)
# print("AMI_funFEM: ", AMI_funFEM/runs)
# print("ARI_funFEM: ", ARI_funFEM/runs)

# Again but round to two decimals
print("AMI_funEnsemble: ", round(AMI_funensemble/runs, 2))
print("ARI_funEnsemble: ", round(ARI_funensemble/runs, 2))
print("AMI_fCUBT: ", round(AMI_fCUBT/runs, 2))
print("ARI_fCUBT: ", round(ARI_fCUBT/runs, 2))
print("AMI_funHDDC: ", round(AMI_funHDDC/runs, 2))
print("ARI_funHDDC: ", round(ARI_funHDDC/runs, 2))
print("AMI_funFEM: ", round(AMI_funFEM/runs, 2))
print("ARI_funFEM: ", round(ARI_funFEM/runs, 2))

AMI_funEnsemble:  0.9615660878048525
ARI_funEnsemble:  0.9437320187384465
AMI_fCUBT:  0.9176366299874136
ARI_fCUBT:  0.9441646076538605
AMI_funHDDC:  0.7898074562805981
ARI_funHDDC:  0.6580028225749059
AMI_funFEM:  0.5952299221028728
ARI_funFEM:  0.5319565275089408
